In [1]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118


In [2]:
!pip install einops timm peft sentencepiece

In [3]:
import os
import glob
import json
import torch
import logging
from transformers import AutoModel, AutoTokenizer
from tqdm import tqdm  # for progress bar

# Disable FLASH_ATTN if needed
os.environ["FLASH_ATTN_DISABLE"] = "1"

# -------------------------------
# 1. Set up logging configuration
# -------------------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger(__name__)

# -------------------------------
# 2. Set up the model and tokenizer for GPU
# -------------------------------
model_path = 'h2oai/h2ovl-mississippi-2b'
model = AutoModel.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True
).eval().cuda()  # Running on GPU

tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True, use_fast=False)
generation_config = dict(max_new_tokens=1024, do_sample=True)

# -------------------------------
# 3. Define the prompt template for structured extraction
# -------------------------------
prompt = ("""<image>\n
<image>\n
Please extract the following information in JSON format:
- Invoice Number
- Company Name
- Address
- Contact Information (Telephone, Email)
- Date (DD/MM/YYYY format)
- Time (HH:MM:SS format)
- Product Details (for each product: Name, ID, Price, Quantity, Total Amount)
- Total Amount
- Cash/Amount Paid
- Change
- Tax/GST
- Discount
Do not include json tags like ```json or ``` in the output. Just return the json object.
If data for a field is not present in the image leave it blank with empty string "".
Always return the output as a valid JSON object with keys corresponding to these fields.
""")

# -------------------------------
# 4. Function to process one folder of images individually and save JSON output
# -------------------------------
def process_folder(input_folder, output_folder):
    image_files = glob.glob(os.path.join(input_folder, "*.png"))
    logger.info(f"Found {len(image_files)} images in {input_folder}.")
    
    for file in tqdm(image_files, desc=f"Processing {os.path.basename(input_folder)}"):
        logger.info(f"Processing {file} ...")
        try:
            # Process one image at a time
            response, history = model.chat(
                tokenizer, file, prompt, generation_config, history=None, return_history=True
            )
            logger.info(f"Raw response for {file}: {response}")
        except Exception as e:
            logger.error(f"Error processing {file}: {e}")
            continue

        try:
            r = response.strip()
            # Remove markdown wrappers if present
            if '```json' in r:
                r = r.split('```json')[1].split('```')[0].strip()
            extracted_data = json.loads(r)
        except Exception as e:
            logger.warning(f"Error parsing JSON for {file}: {e}")
            extracted_data = {"extracted_text": r}
        
        base_name = os.path.splitext(os.path.basename(file))[0]
        output_file = os.path.join(output_folder, f"{base_name}.json")
        try:
            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(extracted_data, f, indent=4)
            logger.info(f"Saved extracted data to {output_file}")
        except Exception as e:
            logger.error(f"Error saving file {output_file}: {e}")
        
        # Display output for each file processed
        print(f"File: {file}")
        print(f"Extracted Data: {extracted_data}\n")

# -------------------------------
# 5. Process dataset folders: train, val, and test and save outputs accordingly
# -------------------------------
dataset_base = "/kaggle/input/find-it-again-dataset/findit2"  # Update with your dataset path
output_base = "/kaggle/working/extracted_jsons"  # Base output folder for JSON files

for split in ["train", "val", "test"]:
    input_folder = os.path.join(dataset_base, split)
    output_folder = os.path.join(output_base, split)
    if os.path.isdir(input_folder):
        os.makedirs(output_folder, exist_ok=True)
        logger.info(f"\nProcessing folder: {input_folder}")
        process_folder(input_folder, output_folder)
    else:
        logger.error(f"Folder {input_folder} does not exist.")


config.json:   0%|          | 0.00/3.61k [00:00<?, ?B/s]

configuration_h2ovl_chat.py:   0%|          | 0.00/3.92k [00:00<?, ?B/s]

configuration_intern_vit.py:   0%|          | 0.00/5.55k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/h2oai/h2ovl-mississippi-2b:
- configuration_intern_vit.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/h2oai/h2ovl-mississippi-2b:
- configuration_h2ovl_chat.py
- configuration_intern_vit.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_h2ovl_chat.py:   0%|          | 0.00/15.1k [00:00<?, ?B/s]

...

File: /kaggle/input/find-it-again-dataset/findit2/test/X51006619780.png
Extracted Data: {'Invoice Number': '001-9822', 'Company Name': 'Yam Fresh', 'Address': 'No.145G, Jalan Rimbunan Raya 1, Laman Rimbunan Kepong, 52100 Kuala Lumpur', 'Contact Information': {'Telephone': '(603) 6243 5520'}, 'Date (DD/MM/YYYY format)': '2016-07-31', 'Time (HH:MM:SS format)': '13:48:37', 'Product Details': [{'Name': 'Yam Fresh Signature-C', 'ID': '1', 'Price': 'RM7.90', 'Quantity': '1', 'Total Amount': 'RM7.90'}], 'Total Amount': 'RM7.90', 'Cash Amount Paid': 'RM10.50', 'Change Given': 'RM2.15', 'Tax/GST': 'RM0.47', 'Discount': ''}



Processing test: 100%|█████████▉| 217/218 [1:18:20<00:22, 22.12s/it]

File: /kaggle/input/find-it-again-dataset/findit2/test/X51005337877.png
Extracted Data: {'Invoice Number': '01H-25029', 'Company Name': 'UROKO JAPANESE CUISINE SDN BHD', 'Address': '22A-1, JALAN 17/54, SECTION 17, 46400 PETALING JAYA, SELANGOR', 'Contact Information': {'Telephone': '03-7932 1023/0191', 'Email': ''}, 'Date': '20/03/2018', 'Time': '6:41:02 PM', 'Product Details': [{'Name': 'KO NABE INANIWA', 'ID': '', 'Price': 24.0, 'Quantity': 1, 'Total Amount': 24.0}, {'Name': 'UDON', 'ID': '', 'Price': 24.0, 'Quantity': 1, 'Total Amount': 24.0}, {'Name': 'GREEN TEA', 'ID': '', 'Price': 1.0, 'Quantity': 1, 'Total Amount': 1.0}, {'Name': 'GYOZA 5pcs', 'ID': '', 'Price': 15.0, 'Quantity': 1, 'Total Amount': 15.0}, {'Name': 'GARLIC CHAHAN', 'ID': '', 'Price': 6.0, 'Quantity': 1, 'Total Amount': 6.0}], 'Total Amount': 46.0, 'Cash/Amount Paid': '', 'Change': '', 'Tax/GST': 3.0, 'Discount': 0.0, 'Net Total': 53.0}



Processing test: 100%|██████████| 218/218 [1:18:38<00:00, 21.65s/it]

File: /kaggle/input/find-it-again-dataset/findit2/test/X51006555784.png
Extracted Data: {'Invoice Number': '7830F715', 'Company Name': 'GARDENIA BAKERIES (M) SDN BHD', 'Address': 'Lot 3, Jalan Pelabur 23/1, 40300 Shah Alam, Selangor', 'Contact Information': {'Telephone': '03- 55423228', 'Email': ''}, 'Date': '30/08/2017', 'Time': '10:51', 'Product Details': [{'Name': 'O.C. WHITE', 'ID': '2.13', 'Price': '5', 'Quantity': '5', 'Total Amount': '10.65'}, {'Name': 'WHOLEMEAL', 'ID': '2.78', 'Price': '5', 'Quantity': '5', 'Total Amount': '13.90'}, {'Name': 'O.C JUMBO', 'ID': '2.97', 'Price': '5', 'Quantity': '5', 'Total Amount': '14.85'}], 'Total Amount': '39.40', 'Cash/Amount Paid': None, 'Change': '53.14', 'Tax/GST': '13.74', 'Discount': None}

